In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append('./../src/')
from manuscript import sankey_side_by_side as sankey
from manuscript import clustering, datasets, inout, export, stats

pd.set_option('display.max_columns', None)

#xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance

# Evaluation of models
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.calibration import calibration_curve
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score


/Users/cag895/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
user = 'general'     # defines top hierarchy of output folder
outfolder = '31_worst_two'    # name of notebook

def dump_table(df, name):
    export.full_frame(
        user, 
        f'{outfolder}/{name}', 
        df, 
        index=True,
        date=True
    )

def dump_figure(name):
    export.image(
        user,
        f'{outfolder}/{name}',
    )

In [2]:
data = pd.read_csv(
    inout.get_material_path('general/30_VAP_flags/data_vap_flags_220919_1403.csv.gz'), 
    index_col=0)

In [20]:
#take the worst value from the first two days 

In [4]:
highbad=['Patient_id','Binary_outcome',#for models later
    'ECMO_flag', 'Intubation_flag', 'Hemodialysis_flag',
       'CRRT_flag', 'Temperature', 'Heart_rate', 
       'Norepinephrine_flag', 'Norepinephrine_rate', 'Respiratory_rate',
        'PEEP',
       'FiO2', 'Plateau_Pressure', 'Lung_Compliance', 'PEEP_changes',
       'Respiratory_rate_changes', 'FiO2_changes',  'ABG_PaCO2',
        'WBC_count', 'Lymphocytes', 'Neutrophils',
       'Creatinine', 
       'Bilirubin', 'CRP', 'D_dimer', 'Ferritin', 'LDH', 'Lactic_acid',
       'Procalcitonin', 'adjudicated','had_nonviral_vap', 'vap_indeterminate_notcured']

lowbad=['Patient_id',
    'Systolic_blood_pressure',
       'Diastolic_blood_pressure', 'Mean_arterial_pressure','Oxygen_saturation', 'Urine_output', 'GCS_eye_opening',
       'GCS_motor_response', 'GCS_verbal_response', 'RASS_score','ABG_pH','ABG_PaO2', 'PaO2FIO2_ratio',
    'Hemoglobin', 'Platelets', 'Bicarbonate', 'Albumin',
]

In [5]:
firsttwodays = data.groupby('Patient_id', as_index=False).head(2)

In [7]:
firsttwohigh = firsttwodays[highbad].groupby('Patient_id', as_index=False).max()
firsttwolow = firsttwodays[lowbad].groupby('Patient_id', as_index=False).min()

In [8]:
worsttwo = pd.merge(firsttwohigh, firsttwolow, on='Patient_id') 

In [9]:
#take only patients who had adjudication data
worsttwo_adjudicated = worsttwo[worsttwo.adjudicated==1]

In [23]:
worsttwo_adjudicated.to_csv("worsttwo_adjudicated8-8-22.csv")

In [11]:
dump_table(worsttwo_adjudicated, 'data_worsttwo_adjudicated.csv.gz')